### Dynamic
Пример отчета по анализу динамики аудитории проектов.

## Описание задания

Задача посчитать аудиторию проекта Kommersant.ru в динамике за 6 месяцев с января по июнь 2020 года.
- Total – Total Internet

Общие параметры:
- Период: 1 января - 30 июня 2020 года
- География: Россия
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline).

список статистик, который будем считать:
- Universe
- Unweighted Reach
- Reach
- Reach%
- Average Daily Reach
- Average Daily Reach%
- Average Weekly Reach
- Average Weekly Reach%


Для расчета данного отчета потребуется сформировать несколько заданий для Responsum


## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot

import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool

import logging

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

Loading BokehJS ...

## Формируем задания

Нам понадобиться сформировать три задания для Responsum:
- расчет динамики аудитории проекта Kommersant.ru
- расчет динамики аудитории Total Internet

### Общие параметры для заданий

Для начала зададим общие параметры

In [2]:
# Задаем параметры
# выбираем тип установки mobile, т.к. в расчетах учавствуют все типы устройств (Web Desktop,Web Mobile,App Online,App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop-pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-01-01'
date_to = '2020-06-30'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, нас интересует города с населением 0+
population = rcats.get_population('Russia0+')

# задаем Возврастные группы, нас интересует 12+
ages = rcats.get_age_groups('12+')

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:19:50
Тип установки: mobile
Начало периода: 2020-01-01
Конец периода: 2020-06-30
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3, 4]
Возрастные группы: [1, 2, 3, 4, 5, 6, 7]
Объектов в media-каталоге: 4716


### Получим ID проектов
Для построения отчета нам необходимо получить идентификатор __Kommerasnt.ru__, для этого воспользуемся блокнотом в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Kommerasnt.ru  site_id = 118

## Задания
Перейдем к формированию заданий

### Задание №1 Аудитория Kommersant в динамике


In [3]:
# Задаем название проекта для отображения в DataFrame
project_name = 'kommersant'

# Задаем медиа фильтр, в нашем случае это ID проекта kommersant.ru
media_filter = "(site = 118)"

# Фильтр по демографии отсутствует
demo_filter = None

# Задаем список статистик для расчета
statistics=["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# Указываем порядок группировки по месяцам
structure =  {
    "usetype": False,
    "date": "month"
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# Получаем результат
df_project = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_project

Задача <829568c3-9f2e-4224-a223-179907244493> поступила в обработку
Расчет задачи [= = = = = ] время расчета: 0:00:16.830922


,prj_name,dt_month,stat_reach_per,stat_reach,stat_unwreach,stat_adr_per,stat_awr_per,stat_awr,stat_adr
0,kommersant,2020-01-01,5.736093,7019.399884,2143.0,0.354604,1.840126,2251.807764,433.937429
1,kommersant,2020-02-01,5.721671,7001.751340,2192.0,0.390250,2.027019,2480.513362,477.558341
2,kommersant,2020-03-01,10.065788,12317.755751,3667.0,0.657147,3.094894,3787.298269,804.167017
3,kommersant,2020-04-01,11.835592,14483.508364,4252.0,0.835995,4.397349,5381.145706,1023.027343
4,kommersant,2020-05-01,8.757022,10716.185827,3140.0,0.571771,3.161773,3869.139766,699.689809
5,kommersant,2020-06-01,6.991850,8556.101369,2660.0,0.431176,2.361311,2889.595171,527.640433


### Задание №2 Total аудитория в динамике

Посчитать данные за полгода по всем проектам довольно сложная задача для Responsum, попробуем обойти это ограничение не большим трюком - воспользуемся технической страницей WebIndex в которую входит вся аудитория.

Получим код страницы

In [4]:
rcats.find_media('Техническая страница', find_in=['site'])

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
2464,1,Техническая страница WEB-Index,101,Техническая страница WEB-Index,214013,mobile_hidden_101,148785,mobile_hidden_101,holding
2465,1,Техническая страница WEB-Index,101,Техническая страница WEB-Index,1001,Техническая страница WEB-Index,1,Техническая страница WEB-Index,holding
2466,1,Техническая страница WEB-Index,101,Техническая страница WEB-Index,72065,hidden_101,30509,hidden_101,holding


Техническая страница WEB-Index - __Site_ID = 101__

In [5]:
# Задаем название проекта для отображения в DataFrame
project_name = 'total_dynamic'

# Задаем техническую страницу WI в качестве медиа фильтра
media_filter = "(site = 101)"

# Фильтр по демографии отсутствует
demo_filter = None

# Задаем список статистик для расчета
statistics=["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# Указываем порядок группировки по месяцам
structure =  {
    "usetype": False,
    "date": "month"
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# Получаем результат
df_total_dynamic = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total_dynamic

Задача <3bdedb06-feb5-4030-b95d-69be230bf34b> поступила в обработку
Расчет задачи [= = = = = = = = ] время расчета: 0:00:26.112111


,prj_name,dt_month,stat_reach_per,stat_reach,stat_unwreach,stat_adr_per,stat_awr_per,stat_awr,stat_adr
0,total_dynamic,2020-01-01,77.149138,94409.321168,19587.0,67.294759,74.214130,90817.678366,82350.272808
1,total_dynamic,2020-02-01,77.238206,94518.316163,19897.0,69.292845,75.386011,92251.739356,84795.379474
2,total_dynamic,2020-03-01,77.938782,95375.627931,20013.0,70.058370,76.026555,93035.587974,85732.171647
3,total_dynamic,2020-04-01,77.735314,95126.638778,19828.0,70.374378,75.820615,92783.574641,86118.879042
4,total_dynamic,2020-05-01,77.906570,95336.209469,19812.0,69.766854,76.068995,93087.523456,85375.436012
5,total_dynamic,2020-06-01,77.949995,95389.349525,19842.0,68.891087,75.750900,92698.262355,84303.738055


### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame

In [6]:
# Объединим наши DataFrame'ы
df_result = pd.concat([df_project, df_total_dynamic])
df_result

,prj_name,dt_month,stat_reach_per,stat_reach,stat_unwreach,stat_adr_per,stat_awr_per,stat_awr,stat_adr
0,kommersant,2020-01-01,5.736093,7019.399884,2143.0,0.354604,1.840126,2251.807764,433.937429
1,kommersant,2020-02-01,5.721671,7001.751340,2192.0,0.390250,2.027019,2480.513362,477.558341
2,kommersant,2020-03-01,10.065788,12317.755751,3667.0,0.657147,3.094894,3787.298269,804.167017
3,kommersant,2020-04-01,11.835592,14483.508364,4252.0,0.835995,4.397349,5381.145706,1023.027343
4,kommersant,2020-05-01,8.757022,10716.185827,3140.0,0.571771,3.161773,3869.139766,699.689809
5,kommersant,2020-06-01,6.991850,8556.101369,2660.0,0.431176,2.361311,2889.595171,527.640433
0,total_dynamic,2020-01-01,77.149138,94409.321168,19587.0,67.294759,74.214130,90817.678366,82350.272808
1,total_dynamic,2020-02-01,77.238206,94518.316163,19897.0,69.292845,75.386011,92251.739356,84795.379474
2,total_dynamic,2020-03-01,77.938782,95375.627931,20013.0,70.058370,76.026555,93035.587974,85732.171647
3,total_dynamic,2020-04-01,77.735314,95126.638778,19828.0,70.374378,75.820615,92783.574641,86118.879042


## Экспорт в Excel

In [7]:
writer = pd.ExcelWriter(rtask.get_excel_filename('kommersant-dynamic'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()